### Imports for Galaxy Subtraction

In [13]:
from pyraf import iraf
import os

#start up iraf to use tasks
iraf.stsdas()
iraf.analysis()
iraf.isophote()

#import fits to read in headers for conveinence
import astropy.io.fits as fits

In [18]:
#specify the input image file
image_file = 'NGC4111_g.fits'

#read in image file's header and get out the x,y max values
hdu = fits.open(image_file)
header = hdu[0].header
x_max = str(header['naxis1'])
y_max = str(header['naxis2'])

#files to save for output. Note that file space can fill up quick when making all these output files.
output_file = 'n4111_g_galsub.fits'
ellipse_table = 'n4111_g_ellipse.tab'
model_file = 'n4111_g_ellipse_model.fits'


#specify the name of the mask region file as well as the mask output file. The mask region file needs to be made
#in ds9. Draw circular regions over all sources near the galaxy that you wish to mask out, then save this file
#in image coordinates.
region_file_name = 'n4111_mask_g.reg'
mask_file_name = 'n4111_mask_g.pl'

In [ ]:
#make the mask for obvious point sources in the galaxy
if os.path.exists(mask_file_name):
    os.remove(mask_file_name)
iraf.mskregions(region_file_name,mask_file_name,dims=x_max +','+y_max)

In [ ]:
#set galaxy parameters

#x and y of galaxy center, measured by hand from image. Needs to be fairly precise, good strategy is to
#centroid an elliptical region around the galaxy
x_0 = 5529.1122
y_0 = 4792.3844

#initial guesses for PA and ellipticity, estimation from image is usually good enough.
init_pa = -60.
init_ellip = 0.8

#initial semi-major axis in pixels, 100 is usually reasonable but might need iteration
init_sma = 100.

#max and min SMA values, max value will need to be played with
max_sma = 900.
min_sma = 20.

### After above values are specified, run next three cells.
Compare galaxy subtracted image with original to verify reasonable subtraction. Will require a lot of iteration, especially on first filter for the galaxy.

In [ ]:

# Define parameters for the ellipse run
# 1. Initial guess of the central X, Y (need to be as accurate as possible)
iraf.ellipse.x0 = x_0
iraf.ellipse.y0 = y_0

iraf.ellipse.ellip0 = init_ellip
iraf.ellipse.pa0 = init_pa
# 3. Initial radius for ellipse fitting (The major axis length of the first
#    elliptical isophote); Can not be too small, and can not be too large
iraf.ellipse.sma0 = init_sma
# 4. The minimum and maximum radius for the ellipse fitting
iraf.ellipse.minsma = min_sma
iraf.ellipse.maxsma = max_sma
# 5. Parameters about the stepsize during the fitting.
#    Unless you know what you what, normally should use log-stepsize instead of
#    linear one; and step=0.05 will generate more isophotes than step=0.1, but
#    may not help if you want a robust surface brightness profile.
iraf.ellipse.linear = "no"
iraf.ellipse.step = 0.05
# 6. Do you want to allow the ellipse to decide the galaxy center during the
#    fitting.  In general, it's a good idea to turn this on.  If the center you
#    provide is accurate enough, ELlipse results will not deviate from it.
iraf.ellipse.recenter = "yes"

iraf.ellipse.hcenter = "no"
iraf.ellipse.hellip = "no"
iraf.ellipse.hpa = "no"

iraf.ellipse.minit = 10
iraf.ellipse.maxit = 300

iraf.ellipse.olthresh = 1.0
iraf.ellipse.dqf = mask_file_name

# 10. Make sure the Interactive Mode is turned off
#iraf.ellipse.controlpar.interactive = "no"


In [ ]:
iraf.ellipse(image_file,ellipse_table)

In [ ]:
#once ellipse fitting ran, write the model to its own fits file
if os.path.exists(model_file):
    os.remove(model_file)
iraf.stsdas.analysis.isophote.bmodel(ellipse_table,model_file)

In [ ]:
#once model written, subtract off the model from the original image
iraf.imarith.operand1 = image_file
iraf.imarith.op = '-'
iraf.imarith.operand2 = model_file
iraf.imarith.result = output_file

if os.path.exists(output_file):
    os.remove(output_file)
iraf.imarith()